In [1]:
import minGPTLightning
from importlib import reload
reload(minGPTLightning)
from minGPTLightning import CharDataModule, GPTConfig, GPT

# Import pytorch lightning
import pytorch_lightning as pl
pl.seed_everything(hash("setting random seeds") % 2**32 - 1)

import wandb
# Use the wandb logger
from pytorch_lightning.loggers import WandbLogger
# login to wandb
wandb.login()

wandb: Currently logged in as: phylliida (use `wandb login --relogin` to force relogin)


True

In [2]:
# setup data

text_data = CharDataModule(text_path="bncsplitwords.txt", block_size=128, train_weight=100, valid_weight=1, test_weight=1, batch_size=256)
text_data.text = text_data.text
text_data.prepare_data()
text_data.setup()

# grab samples to log predictions on
samples = next(iter(text_data.val_dataloader()))

data has 3094841 characters, 61 unique.


In [3]:
# connect to logging weights and biases, see documentation for more info https://docs.wandb.com/integrations/lightning
wandb_logger = WandbLogger(project="text-template")

In [4]:
# create the trainer
# TODO: autoscale batch size
trainer = pl.Trainer(
    logger=wandb_logger,    # W&B integration
    log_every_n_steps=1,   # set the logging frequency
    gpus=-1,                # use all GPUs
    max_epochs=1,           # number of epochs
    deterministic=True,     # keep it deterministic
    gradient_clip_val=1.0,  # gradient clipping
    callbacks=[] # see Callbacks section
    )

GPU available: True, used: True
TPU available: None, using: 0 TPU cores
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]


In [5]:

# setup model
mconf = GPTConfig(vocab_size=text_data.vocab_size, block_size=text_data.block_size,
                  n_layer=4, n_head=4, n_embd=512-(512%4))
model = GPT(mconf, text_data)


In [6]:

print("There will be " + str(len(text_data.train_dataset)) + " training steps")
# fit the model
trainer.fit(model, text_data)



There will be 3034029 training steps



  | Name    | Type       | Params
---------------------------------------
0 | tok_emb | Embedding  | 31.2 K
1 | drop    | Dropout    | 0     
2 | blocks  | Sequential | 12.6 M
3 | ln_f    | LayerNorm  | 1.0 K 
4 | head    | Linear     | 31.2 K
---------------------------------------
12.7 M    Trainable params
0         Non-trainable params
12.7 M    Total params
/home/azureuser/miniconda3/envs/sandbox1/lib/python3.8/site-packages/pytorch_lightning/utilities/distributed.py:49: UserWarning: The dataloader, val dataloader 0, does not have many workers which may be a bottleneck. Consider increasing the value of the `num_workers` argument` (try 6 which is the number of cpus on this machine) in the `DataLoader` init to improve performance.
  warnings.warn(*args, **kwargs)


/home/azureuser/miniconda3/envs/sandbox1/lib/python3.8/site-packages/pytorch_lightning/utilities/distributed.py:49: UserWarning: The dataloader, train dataloader, does not have many workers which may be a bottleneck. Consider increasing the value of the `num_workers` argument` (try 6 which is the number of cpus on this machine) in the `DataLoader` init to improve performance.
  warnings.warn(*args, **kwargs)


wandb: WARNING Step must only increase in log calls.  Step 0 < 1; dropping {'train/loss_step': 4.156198024749756}.
/home/azureuser/miniconda3/envs/sandbox1/lib/python3.8/site-packages/pytorch_lightning/utilities/distributed.py:49: UserWarning: Detected KeyboardInterrupt, attempting graceful shutdown...
  warnings.warn(*args, **kwargs)


1

In [ ]:
# evaluate the model on a test set
trainer.test(datamodule=text_data.test_dataset,
             ckpt_path=None)  # uses last-saved model

# Note: When visiting your run page, it is recommended to use global_step as x-axis to correctly superimpose metrics logged in different stages.
wandb.finish()

Exception: The wandb backend process has shutdown

In [ ]:
_